# Tumor Mutation Classification

In [3]:
import pandas as pd
import boto3
import io

import spacy

nlp = spacy.load('en')

In [4]:
# Set up packages for loading in data
client = boto3.client('s3') #low-level functional API

resource = boto3.resource('s3') #high-level object-oriented API

In [5]:
# Load in training data labels
obj = client.get_object(Bucket='thinkful-capstone', Key='training_variants')
stream = io.BytesIO(obj['Body'].read())
training_variants = pd.read_csv(stream)

In [6]:
print(training_variants.head())
print(training_variants.shape)

   ID    Gene             Variation  Class
0   0  FAM58A  Truncating Mutations      1
1   1     CBL                 W802*      2
2   2     CBL                 Q249E      2
3   3     CBL                 N454D      3
4   4     CBL                 L399V      4
(3321, 4)


In [8]:
# Load in training data text articles
obj = client.get_object(Bucket="thinkful-capstone",Key="training_text")
raw_training_text = obj["Body"].read()
training_text = raw_training_text.decode('utf-8')
print(training_text[:10])

ID||Text
0


In [9]:
# Split text file into list of documents
training_list = training_text.split('||')
training_list = training_list[2:]
print(len(training_list))

3321


In [10]:
# Load training text into dataframe
texts_df = pd.DataFrame(training_list, columns = ['text'])

# Merge text dataframe with labels dataframe
train = pd.concat([training_variants, texts_df], axis=1)
print(train.head())

   ID    Gene             Variation  Class  \
0   0  FAM58A  Truncating Mutations      1   
1   1     CBL                 W802*      2   
2   2     CBL                 Q249E      2   
3   3     CBL                 N454D      3   
4   4     CBL                 L399V      4   

                                                text  
0  Cyclin-dependent kinases (CDKs) regulate a var...  
1   Abstract Background  Non-small cell lung canc...  
2   Abstract Background  Non-small cell lung canc...  
3  Recent evidence has demonstrated that acquired...  
4  Oncogenic mutations in the monomeric Casitas B...  


Now that the data is loaded into a dataframe, let's do some preliminary data cleaning and exploration.

In [11]:
print(train['Class'].value_counts())

7    953
4    686
1    568
2    452
6    275
5    242
3     89
9     37
8     19
Name: Class, dtype: int64


It looks like we are dealing with significant class imbalance here. Class 7 has 953 datapoints, while Class 8 has only 19. Because the labels have been anonymized, we cannot draw any insight about what these classes might signify. 

# Experiment Design

The prevalence of class imbalance has serious implications for our analysis. First and foremost, we must establish our scoring metric. The purpose here is to use the relevant clinical texts to predict the mutation category for each gene/mutation pair. While we want the predictions to be as accurate as possible, simple classification accuracy is not a representative way to judge models that are built on class imbalance, as they may achieve high accuracy by simply predicting the most common class every time. <br>

Given that we are working with a multi-label classifier, the most appropriate scoring metric is Cohen's Kappa coefficient (K). This statistic measures inter-rater agreement for categorical items. Contraty to conventional classification accuracy, the K statistic accounts for the possibility of the agreement occurring by chance. <br>

The Kappa statistic varies from 0 to 1, where 0 = agreement equivalent to chance, and 1 = perfect agreement. We will be choosing models with Kappa statistics closer to 1. We will also look at the precision and recall via the F1 score. Though these are not optimized for multi-label classification, they will be interesting to consider. <br>

We will also try oversampling the lesser-represented categories and apply our machine learning models on the oversampled datasets, judging by their Kappa statistic. Oversampling can be achieved by generating duplicate datapoints or by generating new synthetic datapoints via SMOTE, the Synthetic Minority Oversampling Technique. <br>

I will use various methods of feature generation including classic NLP techniques such as bag-of-words, tf-idf, and n-grams. These methods of feature generation will be applied to both the original and oversampled datasets. They will then be subjected to various machine learning models. Decision trees are known to perform well on unbalanced datasets, so this model may prevail on the original data. However, Naive Bayes is known to perform well on natural langauge, so once the dataset is oversampled it is possible that Naive Bayes will perform the best.

In [14]:
train['spacy_tokens'] = train['text'].apply(lambda x: nlp(x))

def lemmatize(x):
    return [token.lemma_ for token in x
                      if not token.is_stop
                      and not token.is_punct
                      and not token.lemma_ == "-PRON-"
                      and not token.lemma_ == " " or "  "
    ]

train['lemmas'] = train['spacy_tokens'].apply(lambda x: lemmatize(x))
print(train.head())

MemoryError: 

In [ ]:
# Establishing the original dataset

X = train['text']
Y = train['Class']


In [ ]:
train['spacy_tokens'] = train['text'].apply(lambda x: nlp(x))

def lemmatize(x):
    return [token.lemma_ for token in x
                      if not token.is_stop
                      and not token.is_punct
                      and not token.lemma_ == "-PRON-"
                      and not token.lemma_ == " " or "  "
    ]

train['lemmas'] = train['spacy_tokens'].apply(lambda x: lemmatize(x))
print(train.head())